In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [4]:
ncolumns = df.shape[1]
print(f"How many columns are there?: {ncolumns}")

How many columns are there?: 19


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [5]:
df['duration'] = df.apply(lambda x: abs( x['tpep_dropoff_datetime'] - x['tpep_pickup_datetime'] ).total_seconds() / 60.0, axis=1)

In [6]:
stdDurJan = round((df['duration'].std()),2)
print(f"What's the standard deviation of the trips duration in January?: {stdDurJan}")

What's the standard deviation of the trips duration in January?: 42.59


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
cleandDF = df[(df['duration']>= 1.0) & (df['duration']<= 60.0)]

In [8]:
recordsLeft = round((len(cleandDF)/len(df))*100,0)

In [9]:
print(f"What fraction of the records left after you dropped the outliers?: {recordsLeft}%")

What fraction of the records left after you dropped the outliers?: 98.0%


## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)

*Fit a dictionary vectorizer

*Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [10]:
onehotDF = pd.DataFrame()

onehotDF['PULocationID'] = cleandDF['PULocationID'].astype(str)
onehotDF['DOLocationID'] = cleandDF['DOLocationID'].astype(str)
onehotDF['duration'] = cleandDF['duration']
onehotDF = onehotDF.drop_duplicates()

onehotDFDict = onehotDF[['PULocationID','DOLocationID']].drop_duplicates().to_dict(orient='records')
dataDicts = onehotDF[['PULocationID', 'DOLocationID']].to_dict(orient='records')
target = onehotDF['duration'].values

del df
del cleandDF

In [11]:
dicVec = DictVectorizer(sparse=False)
feature_matrix = dicVec.fit_transform(onehotDFDict)
n_feature_cols = feature_matrix.shape[1]

In [12]:
print(f"What's the dimensionality of this matrix (number of columns)?: {n_feature_cols}")

What's the dimensionality of this matrix (number of columns)?: 515
